In [1]:
import numpy as np
from typing import Optional, Tuple

In [2]:
FREQ_OCT = [63,125,250,500,1000,2000,4000,8000]
FREQ_TER = [50,63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,10000]
C = 343.3

In [3]:
_lambda = C/np.array(FREQ_OCT)

In [20]:
_lambda

array([5.44920635, 2.7464    , 1.3732    , 0.6866    , 0.3433    ,
       0.17165   , 0.085825  , 0.0429125 ])

In [4]:
def distancia(posicion1, posicion2):
    posicion1 = np.array(posicion1)
    posicion2 = np.array(posicion2)
    return np.sqrt(((posicion1-posicion2)**2).sum())

In [5]:
class Source:
    def __init__(self, posicion:Tuple[float, float, float], lw:float) -> None:
        self.posicion = posicion
        self.lw = lw
        self.altura = posicion[-1]

    @property
    def get_pos(self):
        return self.posicion[0], self.posicion[2]

In [6]:
class Receiver:
    def __init__(self, posicion:Tuple[float, float, float]) -> None:
        self.posicion = posicion
        self.altura = posicion[-1]

    @property
    def get_pos(self):
        return self.posicion[0], self.posicion[2]
    

In [7]:
class Barrera:
    def __init__(self, x:float, altura:float, espesor:Optional[float]=0) -> None:
        self.x = x
        self.altura = altura
        self.espesor = espesor
    
    @property
    def get_pos(self):
        return self.x, self.altura
    
    @property
    def get_delta_pos(self):
        return self.x + self.espesor, self.altura
    

In [25]:
class AtenuacionBarrera:
    def __init__(self, barrera:Barrera, receiver:Receiver, source:Source) -> None:
        self.receiver = receiver
        self.source = source
        self.barrera = barrera
        self.e = barrera.espesor
        self.c2 = 20

    @property
    def dss(self):
        return distancia(self.source.get_pos, self.barrera.get_pos)
    
    @property
    def dsr(self):
        return distancia(self.barrera.get_delta_pos, self.receiver.get_pos)
    
    @property
    def d(self):
        return distancia(self.receiver.get_pos, self.source.get_pos)
    
    @property
    def a(self):
        return np.abs(self.receiver.altura - self.source.altura)
    
    @property
    def z(self):
        return np.sqrt((self.dss+self.dsr+self.e)**2+(self.a)**2)-self.d
    
    @property
    def kmet(self):
        return np.exp(-(1/2000)*np.sqrt(
            self.dss*self.dsr*self.d/(2*self.z))) if self.z>0 else 1
    
    def _c3_(self, _lambda): 
        return ((1+(5*_lambda/self.e)**2)/(1/3+(5*_lambda/self.e)**2)) if self.e!=0 else 1
    
    def _dz_(self, _lambda):
        return 10*np.log10(3+(self.c2/_lambda)*self._c3_(_lambda)*self.z*self.kmet)
    

In [9]:
barrera = Barrera(30, 3, 0.015)
receptor1 = Receiver((40, 2, 1.5))
fuente1 = Source((25, 2, 1.5), 104)

In [26]:
a = AtenuacionBarrera(barrera, receptor1, fuente1)

In [28]:
a.e

0.015

In [ ]:
a._dz_(_lambda)

array([17.71811177, 20.58312139, 23.53609562, 26.51748246, 29.51339796,
       32.51706703, 35.52636476, 38.54540471])

In [ ]:
barrera.get_pos

(30, 3)